In [1]:
%matplotlib inline
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import tensorflow as tf
from DataUtils import *
import numpy as np
import datetime
import pickle
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors

In [2]:
class ResNet_GAN():
    def __init__(self, **kwargs):
        
        # Default Parameters
        self.show_interval = 10
        self.pre_save_interval = 5000
        self.GD_save_interval = 5000
        
        # Parse Arguments
        
        ### Data Parameters
        self.time = kwargs.get('data', {}).get('time', 39)
        self.vocab = kwargs.get('data', {}).get('vocab', 208024) # Total Number of Words
        self.emb_dim = kwargs.get('data', {}).get('emb_dim', 150) # Embedding Dimension
        self.emb_dir = kwargs.get('data', {}).get('emb_dir', '../data/embedding.npy') # Embedding File Directory
        self.idx_dir = kwargs.get('data', {}).get('emb_dir', '../data/word_index.npy') # Index File Directory
        self.file_tmp = kwargs.get('data', {}).get('file_tmp', '../data/data_t=40.tfrecord') # Data File Template
        self.train_ratio = kwargs.get('data', {}).get('train_ratio', 0.95) # Training Data Proportion
        
        ### Train Parameters
        self.batch_size = kwargs.get('train', {}).get('batch_size', 20)
        self.gen_lr = kwargs.get('train', {}).get('gen_lr', 1e-4) # Generator Learning Rate
        self.dis_lr = kwargs.get('train', {}).get('dis_lr', 5e-5) # Discriminator Learning Rate
        self.dis_n = kwargs.get('train', {}).get('dis_n', 10) # Number of Steps for Training Dis for One Step of Gen
        self.test_mode = kwargs.get('train', {}).get('test_mode', False) 
        
        ### Model Parameters
        self.encode_dim = kwargs.get('model', {}).get('encode_dim', 150) # Dimension of Encoded Vector
        
        self.dis_dim = kwargs.get('model', {}).get('dis_dim', 600)
        self.gen_dim = kwargs.get('model', {}).get('gen_dim', 600)
        self.reload_dir = kwargs.get('model', {}).get('reload_dir', None)
        self.reload_gen = kwargs.get('model', {}).get('reload_gen', None)
        
        # Get Time Stamp
        self.now = datetime.datetime.now()
        
        # Create Tensorflow Session
        self.sess = tf.Session()
        
        # Load Data
        self._load_data()
        
        # Construct Network
        self._construct_model()
        
        # Calculate Loss and Build Optimizer
        self._build_loss()
        self._build_opt()
        
        # Summary
        if not self.test_mode:
            self._build_summary()
        
        # Load Model (If Available)
        self._load_model()
        
    def _load_data(self):
        self.emb_numpy = np.load(self.emb_dir)
        self.emb_tensor = tf.convert_to_tensor(self.emb_numpy, dtype = tf.float32)
        self.emb_index = np.load(self.idx_dir).item() # Index File: word -> index
        self.inv_index = {v: k for k, v in self.emb_index.items()} # Inverse Index: index -> word
        self.train_iter = get_train_test_iter(self.file_tmp, self.train_ratio, self.batch_size, self.sess)
        
        self.next_batch_data = self.train_iter.get_next()
        
        self.real_index = tf.cast(self.next_batch_data['index'], tf.int32)
        self.real_index.set_shape((self.batch_size, self.time + 1))
        self.real_index_list = tf.unstack(self.real_index)
        
        self.global_step = tf.Variable(0, name = 'global_step', trainable = False)
        self.pre_global_step = tf.Variable(0, name = 'pre_global_step', trainable = False)
        self.dis_global_step = tf.Variable(0, name = 'dis_global_step', trainable = False)
        self.bas_global_step = tf.Variable(0, name = 'bas_global_step', trainable = False)
        self.drop_rate = tf.placeholder_with_default(0.3, shape = ())
        self.test_data = tf.placeholder(dtype = tf.int32, shape = (self.batch_size, self.time + 1))

    def _construct_model(self):
        
        self.gen_prob, self.gen_seq, self.pre_prob, self.pre_mask = self.generator(self.real_index)
        self.gen_seq.set_shape((self.batch_size, self.time + 1))
        self.pre_prob.set_shape((self.batch_size, self.time + 1, self.vocab))
        
        self.gen_seq_list = tf.unstack(self.gen_seq)
        self.real_reward, self.real_mask = self.discriminator(self.real_index, reuse = False)
        self.fake_reward, self.fake_mask = self.discriminator(self.gen_seq, reuse = True)
        self.test_reward, self.test_mask = self.discriminator(self.test_data, reuse = True)
        self.fake_baseline = self.baseline(self.gen_seq, reuse = False)
        self.real_baseline = self.baseline(self.real_index, reuse = True)
        
    def _build_loss(self):
        
        
        # weight = tf.range(self.time + 1, 0, - 1, dtype = tf.float32)
        # weight = tf.reshape(weight, (1, self.time + 1))
        
        flatten_x = tf.cast(tf.reshape(self.real_index, [-1]), tf.int32)
        '''
        self.mle_loss = - tf.reduce_sum(
                            tf.multiply(
                                tf.one_hot(flatten_x, self.vocab),
                                tf.log(tf.clip_by_value(tf.reshape(self.pre_prob, [-1, self.vocab]), 1e-20, 1.0))
                            )
                          ) / (self.time + 1) / self.batch_size
        '''
        
        self.mle_loss = - tf.reduce_sum(
                        tf.multiply(
                            tf.reduce_sum(
                            tf.multiply(
                                tf.reshape(tf.one_hot(flatten_x, self.vocab), [self.batch_size, self.time + 1, -1]),
                                tf.log(tf.clip_by_value(self.pre_prob, 1e-20, 1.0))
                            ), 2
                        ), self.pre_mask))
        
        flatten_gen_x = tf.cast(tf.reshape(self.gen_seq, [-1]), tf.int32)
        
        # self.fake_reward_weighted = self.fake_reward * weight
        self.gen_loss = - tf.reduce_mean(tf.multiply(
                            tf.reduce_sum(
                                tf.multiply(
                                    tf.reshape(tf.one_hot(flatten_gen_x, self.vocab), [self.batch_size, -1, self.vocab]),
                                    self.gen_prob)
                                , 2),
                            self.fake_reward - self.real_baseline))
        
        '''\
                        - tf.reduce_mean(tf.multiply(
                            tf.reduce_sum(
                                tf.multiply(
                                    tf.reshape(tf.one_hot(flatten_x, self.vocab), [self.batch_size, -1, self.vocab]),
                                    tf.log(tf.clip_by_value(self.pre_prob, 1e-20, 1.0)))
                                , 2),
                            self.real_reward)) / 2
        '''
        self.bas_loss = tf.reduce_mean(tf.square(self.fake_reward - self.fake_baseline)) + tf.reduce_mean(tf.square(self.real_reward - self.real_baseline))
        
        
        self.real_logit = tf.divide(tf.reduce_mean(self.real_reward, 1), tf.clip_by_value(tf.reduce_sum(self.real_mask, 1), 1, 1e10)) * (self.time + 1)
        self.fake_logit = tf.divide(tf.reduce_mean(self.fake_reward, 1), tf.clip_by_value(tf.reduce_sum(self.fake_mask, 1), 1, 1e10)) * (self.time + 1)
        
        self.dis_loss = - tf.reduce_mean(tf.log(self.real_logit) + tf.log(1 - self.fake_logit))
        
    def _build_opt(self):
        # Collect Variables
        var_gen = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'generator')
        var_dis = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'discriminator')
        var_bas = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'baseline')
        
        # Optimizer
        self.pretrain_opt = tf.train.AdamOptimizer(learning_rate = self.gen_lr).minimize(self.mle_loss, var_list = var_gen, global_step = self.pre_global_step)
        self.dis_opt = tf.train.AdamOptimizer(learning_rate = self.dis_lr).minimize(self.dis_loss, var_list = var_dis, global_step = self.dis_global_step)
        self.gen_opt = tf.train.AdamOptimizer(learning_rate = self.gen_lr).minimize(self.gen_loss, var_list = var_gen, global_step = self.global_step)
        self.bas_opt = tf.train.AdamOptimizer(learning_rate = self.dis_lr).minimize(self.bas_loss, var_list = var_bas, global_step = self.bas_global_step)
    
    def _build_summary(self):
        summary_mle_loss = tf.summary.scalar('MLE_Loss', self.mle_loss)
        summary_gen_loss = tf.summary.scalar('Gen_Loss', self.gen_loss)
        summary_dis_loss = tf.summary.scalar('Dis_Loss', self.dis_loss)
        summary_real_fake_cmp = tf.summary.scalar('Real-Fake', tf.reduce_mean(self.real_logit - self.fake_logit))
        summary_base_loss = tf.summary.scalar('Baseline_Loss', self.bas_loss)
        
        self.summary_pre = tf.summary.merge([summary_mle_loss, ])
        self.summary_GD = tf.summary.merge([summary_gen_loss, summary_dis_loss, summary_real_fake_cmp, summary_base_loss])
        
        self.log_writer = tf.summary.FileWriter( \
                                    f'./log/{str(self.now.month).zfill(2)}_{str(self.now.day).zfill(2)}', \
                                    self.sess.graph)
    
    def _load_model(self):
        self.sess.run(tf.global_variables_initializer())
        self.saver = tf.train.Saver()
        if self.reload_dir != None:
            if self.reload_gen:
                self.saver = tf.train.Saver(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'generator'))
            self.saver.restore(self.sess, self.reload_dir)
            self.saver = tf.train.Saver()
            
    
    def _split_to_list(self, x):
        split = tf.split(x, num_or_size_splits = self.time + 1, axis = 1)
        split = [tf.squeeze(item, axis = [1]) for item in split]
        return split
    
    def _merge_to_one(self, x):
        x = [tf.expand_dims(item, 1) for item in x]
        return tf.concat(x, axis = 1)
    
    # Generator
    def generator(self, x):
        with tf.variable_scope('generator', reuse = tf.AUTO_REUSE):
            
            self.word_embeddings = tf.Variable(self.emb_tensor, name = 'embedding')
            self.gen_unit = tf.nn.rnn_cell.LSTMCell(self.gen_dim, state_is_tuple = False)
            self.proj_layer = tf.layers.Dense(self.vocab)
            
            self.emb_x = tf.nn.embedding_lookup(self.word_embeddings, x)
            
            transpose_x = tf.transpose(tf.nn.embedding_lookup(self.word_embeddings, x), perm = [1, 0, 2])
            self.emb_x_array = tf.TensorArray(tf.float32, size = self.time + 1)
            self.emb_x_array = self.emb_x_array.unstack(transpose_x)
            
            start_token = tf.ones((self.batch_size, ), dtype = tf.int32)
            
            def rnn_unsupervised(step, _x, _h, prob, pred):
                
                _x_emb = tf.nn.embedding_lookup(self.word_embeddings, _x)
                
                _x_emb = tf.expand_dims(_x_emb, 0)
                _h = tf.expand_dims(_h, 0)
                output, state = tf.map_fn(lambda x: self.gen_unit(x[0], x[1]), (_x_emb, _h), dtype = (tf.float32, tf.float32))
                output = tf.squeeze(output, axis = [0])
                state = tf.squeeze(state, axis = [0])
                log_prob = tf.log(tf.clip_by_value(tf.nn.softmax(self.proj_layer(output)), 1e-20, 1.0))
                next_token = tf.cast(tf.reshape(tf.multinomial(log_prob, 1), [self.batch_size]), tf.int32)
                
                next_token = tf.where(tf.equal(_x, 0), tf.zeros(shape = next_token.get_shape(), dtype = tf.int32), next_token)
                
                # next_x = tf.nn.embedding_lookup(self.word_embeddings, next_token)
                prob = prob.write(step, log_prob)
                pred = pred.write(step, next_token)
                return step + 1, next_token, state, prob, pred
            
            def rnn_supervised(step, _x, _h, prob):
                _x = tf.expand_dims(_x, 0)
                _h = tf.expand_dims(_h, 0)
                output, state = tf.map_fn(lambda x: self.gen_unit(x[0], x[1]), (_x, _h), dtype = (tf.float32, tf.float32))
                # output, state = tf.map_fn(lambda x: self.gen_unit(tf.expand_dims(x[0], 0), tf.expand_dims(x[1], 0)), (_x, _h), dtype = (tf.float32, tf.float32))
                output = tf.squeeze(output, axis = [0])
                state = tf.squeeze(state, axis = [0])
                log_prob = tf.nn.softmax(self.proj_layer(output))
                next_x = self.emb_x_array.read(step)
                prob = prob.write(step, log_prob)
                return step + 1, next_x, state, prob
            
            _gen_seq = tf.TensorArray(tf.int32, size = self.time + 1)
            _gen_prob = tf.TensorArray(tf.float32, size = self.time + 1)
            _pre_prob = tf.TensorArray(tf.float32, size = self.time + 1)
            
            '''
            _, _, _, gen_prob, gen_seq = tf.while_loop(
                cond = lambda step, _1, _2, _3, _4: step < self.time + 1,
                body = rnn_unsupervised,
                loop_vars = (tf.constant(0, dtype = tf.int32), # step
                             tf.nn.embedding_lookup(self.word_embeddings, start_token), # initial embedding
                             tf.zeros((self.batch_size, self.gen_dim * 2)), # initial state
                             _gen_prob, _gen_seq),
                return_same_structure = True
            )
            '''
            
            _, _, _, gen_prob, gen_seq = tf.while_loop(
                cond = lambda step, _1, _2, _3, _4: step < self.time + 1,
                body = rnn_unsupervised,
                loop_vars = (tf.constant(0, dtype = tf.int32), # step
                             start_token, # initial token
                             tf.zeros((self.batch_size, self.gen_dim * 2)), # initial state
                             _gen_prob, _gen_seq),
                return_same_structure = True
            )
            
            
            _, _, _, pre_prob = tf.while_loop(
                cond = lambda step, _1, _2, _3: step < self.time + 1,
                body = rnn_supervised,
                loop_vars = (tf.constant(0, dtype = tf.int32), # step
                             tf.nn.embedding_lookup(self.word_embeddings, start_token), # initial embedding
                             tf.zeros((self.batch_size, self.gen_dim * 2)), # initial state
                             _pre_prob),
                return_same_structure = True
            )
            
            pre_mask = self.create_mask(x)
            
            gen_seq = gen_seq.stack()
            gen_seq = tf.transpose(gen_seq, perm = [1, 0])
            
            gen_prob = gen_prob.stack()
            gen_prob = tf.transpose(gen_prob, perm = [1, 0, 2])
            
            pre_prob = pre_prob.stack()
            pre_prob = tf.transpose(pre_prob, perm = [1, 0, 2])
            
            return gen_prob, gen_seq, pre_prob, pre_mask

    def create_baseline_reward(self, r, y):
        r1, _ = tf.split(r, [self.time, 1], 1)
        r2 = tf.concat([tf.ones((self.batch_size, 1)) * y, r1], axis = 1)
        return r2
        
    
    def create_mask(self, x, y = 1):
        cond = 1 - tf.cast(tf.equal(x, 0), tf.float32)
        cond1, _ = tf.split(cond, [self.time, 1], 1)
        cond2 = tf.concat([tf.ones((self.batch_size, 1)), cond1], axis = 1)
        return cond2
    
    # Baseline Network
    def baseline(self, x, reuse = False):
        with tf.variable_scope('baseline', reuse = reuse):
            
            
            
            start_token = tf.ones((self.batch_size, 1), dtype = tf.int32)
            _x = tf.concat([start_token, tf.split(x, [self.time, 1], 1)[0]], axis = 1)
            
            emb_x = tf.nn.embedding_lookup(self.word_embeddings, _x)
            
            
            self.dis_unit = tf.nn.rnn_cell.LSTMCell(self.dis_dim, name = 'forward_unit')
            
            fw_layer = tf.keras.layers.RNN(self.dis_unit, return_sequences = True, name = 'forward_layer')
            # bw_layer = tf.keras.layers.LSTM(self.dis_dim, return_sequences = True, go_backwards = True)
            
            proj_layer = tf.layers.Dense(1, name = 'base_dense_layer')
            
            fw_output = fw_layer(emb_x)
            split_fw_output = self._split_to_list(fw_output)
            fw_logit = []
            for i in range(self.time + 1):
                fw_logit.append(proj_layer(split_fw_output[i]))
            fw_logit = self._merge_to_one(fw_logit)
            # bw_logit = tf.reverse(bw_layer(emb_x), [1])
            
            
            
            reward = tf.squeeze(tf.sigmoid(fw_logit), 2)
            
            mask = self.create_mask(x)
            reward = tf.multiply(reward, mask)
            
            
            return reward
    
    # Discriminator
    def discriminator(self, x, reuse = False):
        with tf.variable_scope('discriminator', reuse = reuse):
            
            emb_x = tf.nn.embedding_lookup(self.word_embeddings, tf.cast(x, tf.int32))
            
            
            self.dis_unit = tf.nn.rnn_cell.LSTMCell(self.dis_dim, name = 'forward_unit')
            
            fw_layer = tf.keras.layers.RNN(self.dis_unit, return_sequences = True, name = 'forward_layer')
            # bw_layer = tf.keras.layers.LSTM(self.dis_dim, return_sequences = True, go_backwards = True)
            
            proj_layer = tf.layers.Dense(1, name = 'dis_dense_layer')
            
            fw_output = fw_layer(emb_x)
            split_fw_output = self._split_to_list(fw_output)
            fw_logit = []
            for i in range(self.time + 1):
                fw_logit.append(proj_layer(split_fw_output[i]))
            fw_logit = self._merge_to_one(fw_logit)
            # bw_logit = tf.reverse(bw_layer(emb_x), [1])
            
            
            
            reward = tf.squeeze(tf.sigmoid(fw_logit), 2)
            
            mask = self.create_mask(x)
            reward = tf.multiply(reward, mask)
            
            return reward, mask
    
    # Train
    def train(self, max_step, stage = 'pretrain-gen'):
        if stage == 'pretrain-gen':
            current_step = self.sess.run(self.pre_global_step)
            try:
                for i in range(max_step):
                    self.sess.run(self.pretrain_opt)
                    current_step += 1
                    if current_step % self.show_interval == 0:
                        summary = self.sess.run(self.summary_pre)
                        self.log_writer.add_summary(summary, current_step)
                    if current_step % self.pre_save_interval == 0:
                        self.saver.save(self.sess, f'./model/Seq_GAN_VarLen_{str(self.now.month).zfill(2)}_{str(self.now.day).zfill(2)}_{str(current_step).zfill(6)}_Gen_Pretrain.ckpt')
            
            except Exception as e:
                print ('Training Process Interrupted! (Pre-train)')
                print (f'Error: {e}')
                print (f'Saving Current Model at Step {current_step}')
                self.saver.save(self.sess, f'./model/Seq_GAN_VarLen_{str(self.now.month).zfill(2)}_{str(self.now.day).zfill(2)}_{str(current_step).zfill(6)}_Gen_Pretrain.ckpt')
            
        else:
            try:
                for i in range(max_step):
                    for j in range(self.dis_n):
                        self.sess.run(self.dis_opt)
                    
                    self.sess.run(self.bas_opt)
                    
                    self.sess.run(self.gen_opt)
                    current_step = self.sess.run(self.global_step)
                    
                    
                    
                    if current_step % self.show_interval == 0:
                        summary = self.sess.run(self.summary_GD)
                        self.log_writer.add_summary(summary, current_step)
                    if current_step % self.GD_save_interval == 0:
                        self.saver.save(self.sess, f'./model/Seq_GAN_VarLen_{str(self.now.month).zfill(2)}_{str(self.now.day).zfill(2)}_{str(current_step).zfill(6)}_GD.ckpt')
            except Exception as e:
                print ('Training Process Interrupted! (Gen-Dis)')
                print (f'Error: {e}')
                print (f'Saving Current Model at Step {current_step}')
                self.saver.save(self.sess, f'./model/Seq_GAN_VarLen_{str(self.now.month).zfill(2)}_{str(self.now.day).zfill(2)}_{str(current_step).zfill(6)}_GD.ckpt')
                
    # Test
    def test(self, mode = 'generation', num_batch = 1):
        if mode == 'generation':
            for i in range(num_batch):
                seq = self.sess.run([self.gen_seq])[0]
                for j in range(self.batch_size):
                    for k in range(self.time + 1):
                        print (self.inv_index[seq[j][k]], end = '/')
                    print ()
    

In [3]:
# config = {'data': {}, 'train': {}, 'model': {}}
# config = {'data': {}, 'train': {}, 'model': {'reload_dir': './model/Seq_GAN_04_02_045000_Gen_Pretrain.ckpt', 'reload_gen': True}}
config = {'data': {}, 'train': {'test_mode': True}, 'model': {'reload_dir': './model/Seq_GAN_VarLen_04_03_010000_GD.ckpt'}}

model = ResNet_GAN(**config)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Use tf.random.categorical instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model/Seq_GAN_VarLen_04_03_010000_GD.ckpt


In [ ]:
model.train(1000000, stage = 'pretrain-gen')

In [ ]:
model.train(1000000, stage = 'gen-dis')

In [4]:
def test(num_batch = 1):
    
    doc = []
    
    for i in range(num_batch):
        seq, logit = model.sess.run([model.gen_seq, model.fake_logit])
        
        for j in range(model.batch_size):
            doc.append([])
            if logit[j] < 0.1:
                continue
            for k in range(model.time + 1):
                print (model.inv_index[seq[j][k]], end = '/')
                doc[-1].append(model.inv_index[seq[j][k]])
                if seq[j][k] == 0:
                    break
            print (logit[j])
    
    return doc

In [12]:
output = test()
# print (output)

it/really/is/no/matter/what/we/deter/fixations/the/three/men/fu/so-called/''/world/./<END>/0.14291003
the/thing/is/the/one/,/so/why/is/what/is/what/?/<END>/0.11334003
avoid/shaft/./<END>/0.6036879
that/is/it/!/<END>/0.35727102
the/story/is/quite/amazing/./<END>/0.26364866
save/goodness/./<END>/0.38372394
did/not/waste/your/money/jolie/retire/./<END>/0.13889363
it/was/a/waste/of/it/./<END>/0.7212733
in/the/problem/,/like/is/that/kind/of/abduction/./<END>/0.16168521
oh/,/if/i/do/i/had/not/work/,/that/was/not/)/./<END>/0.29030794
they/have/to/have/./<END>/0.3442863


In [6]:
# pickle.dump(output, open('../analysis/output_04_04.p', 'wb'))
print (model.emb_tensor)

NameError: name 'model' is not defined

In [4]:
def gen_test(num_batch = 1):
    for i in range(num_batch):
        seq, _, f_reward, f_logit = model.sess.run([model.gen_seq, model.gen_prob, model.fake_reward, model.fake_logit])
        print (seq.shape)
        for j in range(model.batch_size):
            
            for k in range(model.time + 1):
                print (f'{model.inv_index[seq[j][k]]}', end = '/')
                if seq[j][k] == 0:
                    break
            print ()
            print (f_reward[j])
            print (f_logit[j])
            print ()

In [10]:
gen_test()

(20, 40)
so/they/really/it/professes/as/being/it/has/as/the/daughter/of/anorexia/''/and/alexandre/engulfed/as/a/automobile/./<END>/
[2.9796267e-01 2.8136772e-01 9.1834545e-01 6.7280692e-01 3.5223365e-04
 1.6987324e-06 2.6524067e-06 2.9802322e-07 3.8444996e-06 1.2606382e-05
 3.0398369e-06 2.9802322e-07 4.1723251e-07 3.2782555e-07 3.2782555e-07
 4.4703484e-07 8.9406967e-08 5.9604645e-08 2.6822090e-07 1.4901161e-07
 2.3841858e-07 1.4334917e-05 1.3411045e-06 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
0.094385974

one/thing/,/in/the/terms/of/a/story/:/the/movie/gets/true/!/!/!/woodchuck/hardly-distinguishable/woodchuck/sedative/sinus/sedative/bandura/multi-talented/./<END>/
[2.5466615e-01 5.9748799e-02 1.3921022e-02 9.4187260e-04 1.8662214e-04
 2.9733777e-04 4.2021275e-05 3.3289194e-05 1.3557673

In [4]:
def real_test(num_batch = 1):
    for i in range(num_batch):
        seq, f_reward, f_logit = model.sess.run([model.real_index, model.real_reward, model.real_logit])
        print (seq.shape)
        for j in range(model.batch_size):
            
            for k in range(model.time + 1):
                print (f'{model.inv_index[seq[j][k]]}', end = '/')
                if seq[j][k] == 0:
                    break
            print ()
            print (f_reward[j])
            print (f_logit[j])
            print ()

In [5]:
%%time
real_test(num_batch = 10)

(20, 40)
this/movie/attempts/to/make/some/sense/out/of/how/joseph/smith/could/turn/his/``/vision/''/into/a/major/world/religion/./<END>/
[0.34976432 0.9569651  0.9942658  0.9999549  0.9999722  0.9999884
 0.99999774 0.99999917 0.99999934 1.         0.9999994  0.99999285
 0.9999998  1.         1.         1.         1.         0.9999999
 1.         1.         1.         1.         1.         1.
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
0.972036

i/do/not/even/know/if/there/is/an/industry/involved/in/snuff/film/making/,/but/i/hope/it/does/not/./<END>/
[0.37292638 0.8673291  0.24090219 0.03802478 0.25861138 0.70778185
 0.9974371  0.98576254 0.9651123  0.9562638  0.95044434 0.91577196
 0.9132321  0.98740137 0.9998518  0.9999827  0.99998623 0.999975
 0.99999535 0.99997497 0.99999535 0.99999964 0.99990433 0.99954957
 0.         0.         0.         0.         0.         0.

i/do/not/like/hollywood/too/much/,/i/tend/to/like/french/,/italian/,/offbeat/us/or/anything/that/tries/to/communicate/something/sensible.but/this/was/awful/./<END>/
[0.37292638 0.8673291  0.24090219 0.03276584 0.06088144 0.7676438
 0.9996999  0.9998552  0.9993297  0.99538785 0.9981301  0.94519913
 0.9932863  0.9969244  0.9964838  0.9985304  0.99968725 0.99988186
 0.9999992  0.9999999  1.         1.         1.         1.
 0.999999   0.9999999  1.         1.         1.         0.9999976
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
0.87951094

my/husband/and/i/were/captivated/by/it/when/it/first/aired/in/the/us/and/watched/every/episode/./<END>/
[0.14672375 0.23798198 0.9955907  0.9998337  0.99980944 0.9979466
 0.9990972  0.9998824  0.9999975  0.99998426 0.99999505 0.9999888
 0.9999999  1.         1.         1.         0.99999994 0.9999986
 0.99999905 0.99999726 0.9999998  0.         0.         0.
 0.         0.         0.       

(20, 40)
and/there/is/a/little/picture/,/just/for/your/imagination/,/how/a/device/like/this/should/look/or/can/look/./<END>/
[0.30459794 0.1819295  0.92389894 0.9159479  0.8984033  0.9374578
 0.7080804  0.4602879  0.614275   0.73314565 0.6972356  0.99518603
 0.9941262  0.99991643 0.999806   0.99941444 0.99998486 0.99999994
 1.         0.9999999  1.         1.         0.9999988  1.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
0.84848726

in/addition/you/are/never/more/than/a/few/minutes/away/from/a/gory/killing/./<END>/
[0.2652092  0.18123683 0.99020106 0.9870409  0.95976365 0.9947295
 0.98547626 0.9720385  0.9968344  0.9989913  0.99972963 0.9997958
 0.9999964  0.9996685  0.9993557  0.9962374  0.99945074 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.

In [5]:
print (tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'discriminator'))

[<tf.Variable 'discriminator/forward_unit/kernel:0' shape=(800, 2400) dtype=float32_ref>, <tf.Variable 'discriminator/forward_unit/bias:0' shape=(2400,) dtype=float32_ref>, <tf.Variable 'discriminator/dis_dense_layer/kernel:0' shape=(600, 1) dtype=float32_ref>, <tf.Variable 'discriminator/dis_dense_layer/bias:0' shape=(1,) dtype=float32_ref>]


In [11]:
doc = pickle.load(open('../data/doc.p', 'rb'))

In [19]:
def find_similar_sent(num_batch = 1):
    
    for i in range(num_batch):
        seq = model.sess.run([model.gen_seq])[0]
        
        for k in range(model.time + 1):
            print (model.inv_index[seq[0][k]], end = '/')
        
        print ()
        
        similarity = -1
        index = -1
        
        for j in tqdm(range(len(doc))):
            cnt = 0
            
            for k in range(min(len(doc[j]) - 1, 39)):
                if (model.emb_index[doc[j][k + 1]] in seq[0]) and (doc[j][k + 1] != '<END>'):
                    cnt += 1
            if cnt / min(len(doc[j]) - 1, 39) > similarity:
                similarity = cnt / min(len(doc[j]) - 1, 39)
                index = j
        
        for k in range(min(len(doc[index]) - 1, 39)):
            print (doc[index][k + 1], end = '/')
        
        print ()

In [20]:
find_similar_sent()

  0%|          | 1082/634943 [00:00<00:58, 10813.05it/s]

a/nagging/f/setup/./semper/drunk.downbeat/./<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/<END>/


100%|██████████| 634943/634943 [00:44<00:00, 14377.78it/s]

a/./<END>/


In [35]:
def test_sent(sent = ['i', 'will']):
    
    length = len(sent) + 1
    
    for i in range(len(sent), model.time + 1):
        sent.append('<END>')
    
    doc = np.zeros((model.batch_size, model.time + 1), dtype = np.int32)
    
    for i in range(model.time + 1):
        doc[:, i] = model.emb_index[sent[i]]
    print (doc[0])
    result = model.sess.run(model.test_reward, feed_dict = {model.test_data: doc})[0]
    return result, np.sum(result) / length

In [45]:
test_sent(['yeah', '.'])

[8088   19    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


(array([0.15591392, 0.9988781 , 0.16909182, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       dtype=float32), 0.4412946303685506)

In [ ]:
from sklearn.manifold import TSNE
def test_t_sne(test_n = 15):
        nodes = np.zeros((model.batch_size * 2 * test_n, model.encode_dim))
        for i in range(test_n):
            x = model.sess.run(model.fake_vec)
            # x = np.reshape(x, (model.batch_size, model.encode_dim))
            nodes[i * model.batch_size: (i + 1) * model.batch_size, :] = x
            x = model.sess.run(model.real_vec)
            # x = np.reshape(x, (model.batch_size, model.encode_dim))
            nodes[(i + test_n) * model.batch_size: (i + 1 + test_n) * model.batch_size, :] = x
        nodes_emb = TSNE(n_components=2).fit_transform(nodes)
        plt.scatter(nodes_emb[:model.batch_size * test_n, 0], nodes_emb[:model.batch_size * test_n, 1], c = 'red', label = 'fake', alpha = 0.3)
        plt.scatter(nodes_emb[model.batch_size * test_n:, 0], nodes_emb[model.batch_size * test_n:, 1], c = 'blue', label = 'real', alpha = 0.3)
        plt.legend()
        plt.show()

In [ ]:
test_t_sne()

In [19]:
model.sess.run([model.real_logit, model.fake_logit, model.fake_reward])

[array([0.758016  , 0.2945411 , 0.81416667, 0.93891776, 0.9163955 ,
        0.7524428 , 0.9723311 , 0.9694891 , 0.98420054, 0.9713954 ,
        0.86335415, 0.9874011 , 0.3663145 , 0.8702861 , 0.97608095,
        0.96792966, 0.24847725, 0.8600267 , 0.9772145 , 0.84713376],
       dtype=float32),
 array([0.04581574, 0.1114279 , 0.11969958, 0.04087158, 0.38883203,
        0.05364672, 0.05690962, 0.09124173, 0.03349488, 0.03549676,
        0.4377616 , 0.07964477, 0.04734786, 0.02419231, 0.5625976 ,
        0.6606058 , 0.14276339, 0.26952118, 0.10039376, 0.04364331],
       dtype=float32),
 array([[5.74482024e-01, 1.32345408e-01, 2.13622451e-02, 4.84788418e-03,
         1.35600567e-05, 6.55651093e-07, 1.19209290e-07, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.

In [18]:
model.sess.run(model.bas_loss)

0.1402963

In [ ]:
x = model.sess.run(model.real_vec)
y = model.sess.run(model.real_vec)
z = (x + y) / 2.
print (z.shape)
z = tf.convert_to_tensor((x + y) / 2, dtype = tf.float32)
model.sess.run(model.decoder(z))

In [33]:
model.sess.run(model.pre_global_step)

0

In [8]:
model.saver.save(model.sess, f'./model/Seq_GAN_{str(model.now.month).zfill(2)}_{str(model.now.day).zfill(2)}_5000_GD.ckpt')

'./model/Seq_GAN_04_02_5000_GD.ckpt'

In [7]:
model.saver = tf.train.Saver()